### M6 EDA Exercise


##### Exercises
<br>

<ol>
  <li>Import the Hep Dataset and perform EDA as outlined below to the text column.</li>
    
    
                Plot the top 10 highest frequency words.
                Calculate the normalised frequency for top 10 most frequently occurring words.
                Create a word cloud for the top 100 words (in terms of frequency)
                Calculate the lexical diversity for the text
                Show keyword in context for the top 10 words.
                Generate collocations

     
</ol>

In [ ]:
# Import libraries used
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.data.path.append("../pre_course/nltk_data")
from collections import Counter
from nltk import ngrams
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import collections

In [ ]:
hep = pd.read_pickle("../data/Hep_Dataset.pkl")

In [ ]:
# Function to change text column to string
def change_to_string(ptext):
    return " ".join(ptext)

In [ ]:
hep['Text'] = hep['Text'].apply(change_to_string)

In [ ]:
#keep a copy of the df 
hep2 = hep.copy()

### Plot the top 10 highest frequency words.

In [ ]:
# Some utility preprocessing functions for use later.

def remove_punct(ptext):
    
    for each_punctuation_mark in string.punctuation:
        ptext = ptext.replace(each_punctuation_mark, "")
        
    return ptext


stop_words = set(stopwords.words('english'))


def clean_stopwords(token):
    return [item for item in token.split(" ") if item not in stop_words and item != '']


In [ ]:
#lowercase
hep["Text"] = hep["Text"].str.lower()

In [ ]:
#remove punctuation
hep['Text'] = hep['Text'].apply(remove_punct)

In [ ]:
#remove stopwords
hep['Text_nostopwords'] = hep['Text'].apply(clean_stopwords)

In [ ]:
hep.loc[5, 'Text_nostopwords']

In [ ]:
alltext = []
for item in hep['Text_nostopwords']:
    alltext.extend(item)

In [ ]:
counter=collections.Counter(alltext)
print(dict(counter))

In [ ]:
sort_order = sorted(dict(counter).items(), key=lambda x: x[1], reverse=True)
token=[]
count =[]
for i in sort_order:
    token.append(i[0])
    count.append(i[1])

In [ ]:
def plotall(px, py):
    
    plt.xticks(fontsize=12, rotation=90)
    plt.ylabel('Frequency')
    plt.plot(px, py)
    plt.show()

In [ ]:
 
plotall(token[:10], count[:10])

### Calculate the normalised frequency for top 10 most frequently occurring words.

In [ ]:
# Get total tokens in text column
total = sum(dict(counter).values())

In [ ]:
for num in range(10):
    print (token[num], "-", count[num]/total)

### Create a word cloud for the top 100 words (in terms of frequency)

In [ ]:
# Join all the text data
text = " ".join(hep['Text'])

# The text string is then passed to the wordcloud function:
wordcloud = WordCloud(max_font_size=50, 
                      max_words=100, 
                      background_color="white").generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Calculate the lexical diversity for the text

In [ ]:
print ('Lexical Diversity:',len(dict(counter).keys())/total)

### Show keyword in context for the top 10 words.
            

In [ ]:
len(text)

In [ ]:
tokens = text.split(" ")
textList = nltk.Text(tokens)

In [ ]:
textList.concordance('model')

### Generate collocations

In [ ]:
def generate_collocations(tokens):
    '''
    Given list of tokens, return collocations.
    '''
    ignored_words = nltk.corpus.stopwords.words('english')
    
    bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
    bigramFinder.apply_word_filter(lambda word: len(word) < 3 or word.lower() in ignored_words)
    
    bigram_freq = bigramFinder.ngram_fd.items()
    
    bigramFreqTable = (pd.DataFrame(list(bigram_freq), columns=['bigram','freq'])
                       .sort_values(by='freq', ascending=False))
   
    return bigramFreqTable


In [ ]:
generate_collocations(textList).head(50)